In [2]:
import numpy as np
from PIL import Image

# Load the image
img = Image.open("your_image.jpg")

# Convert the image to a numpy array
img_array = np.array(img)

# Flatten the array to a 1D array
img_1d = img_array.flatten()

# Print the shape of the 1D array
print(img_1d.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'your_image.jpg'

In [3]:
import os
import numpy as np
import cv2
# import keras
# import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from keras.preprocessing import image
from matplotlib import pyplot as plt
# from keras.models import Sequential
# from keras.layers import Dense, Input, Dropout, Flatten, Layer, Conv2D, Lambda, MaxPooling2D
# from keras import regularizers
# from keras.losses import categorical_crossentropy
# import keras.utils
# from keras import backend as K
import tarfile
from sklearn.metrics import accuracy_score

X = []
Y = []
class_name=[]
path="archive/lfw-funneled.tgz"
tar = tarfile.open(path, "r:gz")
counter=0
for tarinfo in tar:
    tar.extract(tarinfo.name)
    if tarinfo.name[-4:]==".jpg":
        people_name=os.path.basename(tarinfo.name).split('.')[0].rsplit('_', 1)[0]
        image = cv2.imread(tarinfo.name)
        X.append(np.array(image))
        Y.append(people_name)
    if tarinfo.isdir():
        class_name.append(os.path.basename(tarinfo.name))
        pass
tar.close()
class_name=class_name[1:]
X= np.array(X)
print("LFW images have been fetched")


FileNotFoundError: [Errno 2] No such file or directory: 'archive/lfw-funneled.tgz'

In [ ]:
print(X[0].flatten().shape)
X = X.reshape(X.shape[0],-1)
# for i in range(len(X)):
#     X[i].reshape(X[i].flatten().shape[0])
#     X[i] = X[i].flatten()
    
print(X.shape)


(187500,)
(13233, 187500)
['George_HW_Bush' 'George_HW_Bush' 'George_HW_Bush' ... 'Leticia_Dolera'
 'Joseph_Fiennes' 'Colin_Prescot']


In [ ]:
import numpy as np

def convert_classes_to_numbers(arr):
    unique_values = np.unique(arr)
    class_to_num = {class_name: i for i, class_name in enumerate(unique_values)}
    return np.vectorize(class_to_num.get)(arr)


In [ ]:
y = np.array(Y)
y = convert_classes_to_numbers(y)
print(np.unique(y).shape)

(5749,)


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


##Helper functions. Use when needed. 
def show_orignal_images(pixels):
	#Displaying Orignal Images
    fig, axes = plt.subplots(6, 10, figsize=(11, 7),subplot_kw={'xticks':[], 'yticks':[]})
    for i, ax in enumerate(axes.flat):
        ax.imshow(np.array(pixels)[i].reshape(64, 64), cmap='gray')
    plt.show()

def show_eigenfaces(pca):
	#Displaying Eigenfaces
    fig, axes = plt.subplots(3, 8, figsize=(9, 4),subplot_kw={'xticks':[], 'yticks':[]})
    for i, ax in enumerate(axes.flat):
        ax.imshow(pca.components_[i].reshape(64, 64), cmap='gray')
        ax.set_title("PC " + str(i+1))
    plt.show()



## Step 1: Read dataset and visualize it
df = pd.read_csv("face_data.csv")
targets = df["target"]
pixels = df.drop(["target"],axis=1)

print (np.array(pixels).shape)

# show_orignal_images(pixels)
## Step 2: Split Dataset into training and testing
x_train, x_test, y_train, y_test = train_test_split(X, y)

## Step 3: Perform PCA.
pca = PCA(n_components=150).fit(x_train)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');
plt.show()

# show_eigenfaces(pca)

## Step 4: Project Training data to PCA
print("Projecting the input data on the eigenfaces orthonormal basis")
Xtrain_pca = pca.transform(x_train)

##############

## Step 5: Initialize Classifer and fit training data
clf = SVC(kernel='rbf',C=1000,gamma=0.001)
clf = clf.fit(Xtrain_pca, y_train)

param_grid = {'kernel' : ('poly', 'rbf'), 'C':[0.01, 0.1, 0.5, 1, 10,50,100,1000],'gamma':[0.1,0.01,0.001,0.0001,0.00001,0.000001]}
gs = GridSearchCV(estimator=SVC(kernel='rbf'),
param_grid=param_grid,
refit=True, # choose best model and refit to entire data
cv=10,
n_jobs=None)
gs.fit(Xtrain_pca,y_train)

best_params = gs.best_params_
print('Best Params:', best_params)
bestModel = SVC(kernel=best_params['kernel'],C=best_params['C'])
bestModel.fit(Xtrain_pca,y_train)
print('Accuracy of SVC on training set: {:.2f}'.format(bestModel.score(Xtrain_pca,y_train)*100))
print('Accuracy of SVC on test set: {:.2f}'.format(bestModel.score(x_test,y_test)*100))


## Step 6: Perform testing and get classification report
# print("Predicting people's names on the test set")
# t0 = time()
# Xtest_pca = pca.transform(x_test)
# y_pred = clf.predict(Xtest_pca)
# print("done in %0.3fs" % (time() - t0))
# print(classification_report(y_test, y_pred))
# print(accuracy_score(y_test,y_pred))




(400, 4096)


NameError: name 'y' is not defined